# Studying network construction: Effects of number of edges

The main objective is to check how the network cardinality measures (basically number of edges and degree of each node) is effecting the ratings.

Two examples are following.

 - A network construction with a maximum of edges.
 - A random network construction by edge probability.

### Import statements and Random fix

In [1]:
import random
import numpy as np

random.seed(321)
np.random.seed(1234)

In [2]:
import networkx as nx
from dfg_rating.model import factory

from dfg_rating.model.network.base_network import BaseNetwork
from dfg_rating.model.network.simple_network import RoundRobinNetwork

from dfg_rating.model.rating.ranking_rating import LeagueRating
from dfg_rating.model.rating.controlled_trend_rating import ControlledTrendRating, ControlledRandomFunction
from dfg_rating.model.rating.elo_rating import ELORating

from dfg_rating.model.forecast.true_forecast import LogFunctionForecast

from dfg_rating.model.evaluators.accuracy import RankProbabilityScore

import dfg_rating.viz.jupyter_widgets as DFGWidgets

In [3]:
true_rating = ControlledTrendRating(
    starting_point=ControlledRandomFunction(distribution='normal', loc=1000, scale=100),
    delta=ControlledRandomFunction(distribution='normal', loc=0, scale=3),
    trend=ControlledRandomFunction(distribution='normal', loc=0, scale=20/365),
    season_delta=ControlledRandomFunction(distribution='normal', loc=0, scale=10)
)

## Creating Networks

#### standard_n := Standard 10-Round-Robin league

In [11]:
standard_n: BaseNetwork = factory.new_network(
    'multiple-round-robin',
    teams=18,
    seasons=5,
    league_temas=18,
    league_promotion=0,
    days_between_rounds=3,
    true_rating=true_rating
)
standard_n.create_data()
standard_n.play()

Season 0
Relegation candidates {4: 8.0, 13: 8.0, 8: 13.0, 9: 15.0, 17: 17.0, 2: 19.0}
Relegation teams []
Season 1
Relegation candidates {4: 5.0, 13: 14.0, 8: 14.0, 0: 14.0, 12: 16.0, 5: 20.0}
Relegation teams []
Season 2
Relegation candidates {0: 7.0, 2: 11.0, 4: 11.0, 14: 13.0, 13: 16.0, 15: 18.0}
Relegation teams []
Season 3
Relegation candidates {11: 7.0, 2: 7.0, 4: 11.0, 9: 17.0, 0: 18.0, 8: 20.0}
Relegation teams []
Season 4
Relegation candidates {2: 10.0, 9: 11.0, 8: 12.0, 4: 15.0, 6: 15.0, 13: 15.0}
Relegation teams []
Multiple seasons is played when creating data


#### max_n := Network with a maximum number of edges

In [ ]:
class EdgeCappedNetwork(RoundRobinNetwork):
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.max_edges = kwargs.get("max_edges", (self.n_teams * (self.n_teams - 1)))
    
    def fill_graph(self):
        if self.data is None:
            graph = nx.MultiDiGraph()
        else:
            graph = self.data
            
        current_edges = 0
        n_list = [n for n in range(self.n_teams)]
        graph.add_nodes_from([n for n in range(self.n_teams)])
        
        while current_edges < self.max_edges:
            u = random.choice(n_list)
            v = random.choice(n_list)
            if u==v or graph.has_edge(u,v):
                continue
            else:
                n_round = max(graph.degree[v], graph.degree[u])
                graph.add_edge(u,v, season=0, round=n_round, day=self.days_between_rounds * n_round)
                current_edges += 1
                
        self.data = graph          
        

In [ ]:
max_n = EdgeCappedNetwork(
    teams=200,
    days_between_rounds=3,   
    max_edges=10000,
    true_rating=true_rating
)
max_n.create_data()
max_n.play()

#### random_n := Network with edge probability

We create networks with edge probability from 0.01 to 1.00 (Completed)

In [4]:
class RandomNetwork(RoundRobinNetwork):
    """
    Choses each of the possible [n(n-1)]/2 edges with probability p.
    """
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.edge_probability = kwargs.get("edge_probability", 1)
    
    def fill_graph(self):
        if self.data is None:
            graph = nx.MultiDiGraph()
        else:
            graph = self.data
            
        current_edges = 0
        n_list = [n for n in range(self.n_teams)]
        graph.add_nodes_from([n for n in range(self.n_teams)])
        
        for u in range(self.n_teams):
            for v in range(u+1, self.n_teams):
                if random.random() < self.edge_probability:
                    n_round = max(graph.degree[v], graph.degree[u])
                    graph.add_edge(u,v, season=0, round=n_round, day=self.days_between_rounds * n_round)
        self.data = graph 

In [ ]:
random_n = RandomNetwork(
        teams=500,
        days_between_rounds=3,   
        edge_probability=1,
        true_rating=true_rating
    )
random_n.create_data()
random_n.play()
networks_list = [random_n]

In [ ]:
networks_list = []
for prob in np.arange(0.01, 1.01, 0.01):
    random_n = RandomNetwork(
        teams=128,
        days_between_rounds=3,   
        edge_probability=prob,
        true_rating=true_rating
    )
    random_n.create_data()
    random_n.play()
    networks_list.append(random_n)

In [ ]:
print(len(networks_list[99].data.edges))
networks_list[99].edge_probability

## Explore networks

In [ ]:
network_explorer = DFGWidgets.NetworkExplorer(network=standard_n)

In [ ]:
network_explorer.run('inline')

### ELO Rating to the networks with RPS evaluation

In [12]:
elo = ELORating(**{
    "trained": True,
})
elo_forecast = LogFunctionForecast(
    outcomes=['home', 'draw', 'away'],
    coefficients=[-0.9, 0.3],
    beta_parameter=0.006
)
rps = RankProbabilityScore(
    outcomes=['home', 'draw', 'away'],
    forecast_name='elo_forecast'
)

In [13]:
for n in [standard_n]:
    n.add_rating(rating=elo, rating_name='elo_rating')
    n.add_forecast(forecast=elo_forecast, forecast_name='elo_forecast', base_ranking="elo_rating")
    n.add_evaluation(rps, "elo_rating")

## Results

In [14]:
ratings_explorer = DFGWidgets.RatingsEvaluation(network=standard_n)

In [15]:
ratings_explorer.run('inline', port=8051)

Selected teams : None
Selected teams : None
Selected teams : None
